## How to call functions with model generated arguments

In our next example, we'll demonstrate how to execute functions whose inputs are model-generated, and use this to implement an agent that can answer questions for us about a database. For simplicity we'll use the [Chinook sample database](https://github.com/lerocha/chinook-database).

*Note:* SQL generation can be high-risk in a production environment since models are not perfectly reliable at generating correct SQL.

## Install PowerShellAI

In [ ]:
$module = 'PowerShellAI'
$moduleExists = gmo -list $module 
if($null -eq $moduleExists) {
    Install-Module -Name $module -Scope CurrentUser 
}

Import-Module PowerShellAI -Force